In [ ]:
import os
import sys

app_dir = os.path.abspath(os.path.join(os.path.dirname(os.path.abspath(__name__)), "../app"))
sys.path.append(app_dir)
from utils import helpers as h

h.record_builder("2024-25")

2023-24 Western
                        wins  losses  win_%
team_name                                  
Denver Nuggets            64      30   0.68
Oklahoma City Thunder     63      29   0.68
Minnesota Timberwolves    65      33   0.66
Dallas Mavericks          63      41   0.61
Los Angeles Clippers      53      35   0.60
Phoenix Suns              49      37   0.57
New Orleans Pelicans      50      38   0.57
Sacramento Kings          47      37   0.56
Los Angeles Lakers        49      39   0.56
Golden State Warriors     46      37   0.55
Houston Rockets           41      41   0.50
Utah Jazz                 31      51   0.38
Memphis Grizzlies         27      55   0.33
San Antonio Spurs         22      60   0.27
Portland Trail Blazers    21      61   0.26 

2023-24 Eastern
                     wins  losses  win_%
team_name                               
Boston Celtics         80      21   0.79
New York Knicks        57      38   0.60
Milwaukee Bucks        51      37   0.58
Orlando Magic